# Simulation template (less features)

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963). Here the simulation is run with fewer lattice sites

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 740 ms, sys: 249 ms, total: 989 ms
Wall time: 573 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_LESS_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = [0, 1, 3, 50, 51, 53]
shuffle_features = False
random_state = 936                   

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = [0, 1, 3, 50, 51, 53]
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh1/periodic_100_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh1/periodic_less_100_6561"
random_state = 784


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656100 entries, 0 to 656099
Columns: 108 entries, id to feat99
dtypes: float64(103), int32(3), object(2)
memory usage: 538.1+ MB
CPU times: user 5.11 s, sys: 505 ms, total: 5.61 s
Wall time: 5.61 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,666,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  100
n_hamiltonians:  6561
n_ts:  2
CPU times: user 145 µs, sys: 19 µs, total: 164 µs
Wall time: 85.6 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.846822130772748
% val:  0.0
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  5556
number of val hamiltonians:  0
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:01<02:59,  1.81s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [00:03<02:51,  1.75s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [02:42<00:00,  1.65s/it]

running experiments: 100%|██████████| 100/100 [02:42<00:00,  1.63s/it]

CPU times: user 2min 40s, sys: 2.81 s, total: 2min 43s
Wall time: 2min 42s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
0,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,0,test,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,0.141421,0.141142,...,0.114412,0.108967,0.123928,0.119406,0.131490,0.127962,0.136978,0.134500,0.140306,0.138916
2,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,0.141421,0.140306,...,0.043702,0.026500,0.075777,0.060214,0.103092,0.090145,0.123928,0.114412,0.136978,0.131490
3,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,-0.100000,0.100000,...,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000
4,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,0,test,0.000509,-0.008372,...,-0.082713,-0.089753,-0.067684,-0.075347,-0.051587,-0.059753,-0.034677,-0.043217,-0.017220,-0.026000
5,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,0,test,0.001204,-0.016530,...,-0.134123,-0.138686,-0.118757,-0.127445,-0.095929,-0.108196,-0.067073,-0.082149,-0.034003,-0.050940
6,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,0,test,-0.141421,-0.138980,...,0.043375,0.067830,-0.009222,0.017385,-0.060524,-0.035502,-0.103326,-0.083403,-0.131616,-0.119589
7,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,-0.000826,0.025688,...,0.134753,0.124324,0.141088,0.140407,0.127608,0.136771,0.096206,0.113925,0.051292,0.075079
8,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,0.141417,0.136684,...,-0.115093,-0.131915,-0.061266,-0.091040,0.007717,-0.027643,0.074791,0.042592,0.123363,0.102291
9,0,H_-2_-2_0.509296.dat,-2.0,-2.0,0.509296,999,1,test,0.001302,0.036430,...,0.082068,0.050848,0.127402,0.108132,0.141218,0.138667,0.120099,0.134896,0.069268,0.097755


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 55.4 ms, sys: 73 µs, total: 55.4 ms
Wall time: 55 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99
39984,399,H_-1.8_1.75_0.134419.dat,-1.80,1.75,0.134419,0,1,test,0.141421,-0.115366,...,-0.141421,0.115366,0.043523,-0.113444,0.114523,-0.045254,-0.114302,0.141412,-0.043880,-0.042144
41017,410,H_-1.75_-1.75_0.509296.dat,-1.75,-1.75,0.509296,999,1,test,0.002245,-0.019950,...,0.135177,-0.139320,0.120594,-0.128902,0.098434,-0.110384,0.070089,-0.084931,0.037340,-0.054142
483518,4835,H_0.95_0.8_-0.0137367.dat,0.95,0.80,-0.013737,0,0,test,0.000393,0.047200,...,0.082807,-0.116545,-0.018115,-0.030120,-0.109217,0.072632,-0.141117,0.136013,-0.096522,0.125666
127309,1273,H_-1.25_0.9_0.0011202.dat,-1.25,0.90,0.001120,0,0,val,-0.002485,-0.031845,...,0.081103,0.055228,-0.126884,-0.111117,0.141277,0.139517,-0.120719,-0.133403,0.070297,0.094286
273417,2734,H_-0.35_1.05_1.00201.dat,-0.35,1.05,1.002007,1,1,val,-0.001596,0.041099,...,0.133998,-0.115994,-0.141234,0.137631,0.128633,-0.139938,-0.097967,0.122591,0.053541,-0.088027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444480,4444,H_0.7_1.5_1.00501.dat,0.70,1.50,1.005012,1,1,val,-0.141393,0.114326,...,0.040995,0.043841,0.130420,-0.075654,0.125270,-0.140288,0.029280,-0.103192,-0.087942,0.008734
392912,3929,H_0.4_0.05_-0.000382047.dat,0.40,0.05,-0.000382,0,0,val,0.100000,-0.100000,...,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000,0.100000,-0.100000
428535,4285,H_0.6_1.65_1.0042.dat,0.60,1.65,1.004203,1,1,train,0.141421,-0.047100,...,0.043448,0.112267,-0.131392,-0.005296,0.124057,-0.105515,-0.026761,0.139811,-0.089940,-0.072724
370518,3705,H_0.25_1_1.00357.dat,0.25,1.00,1.003567,1,1,train,0.100000,-0.100000,...,-0.100000,0.100000,0.100000,-0.100000,-0.100000,0.100000,0.100000,-0.100000,-0.100000,0.100000


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.08474317939338516
% val:  0.7620789513793629
% test:  0.15317786922725193
% train + val + test:  1.0


number of train hamiltonians:  556
number of val hamiltonians:  5000
number of test hamiltonians:  1005
total number of hamiltonians:  6561


train ids:  [1621, 4691, 3502, 4297, 4971, 1774, 3314, 3257, 2998, 1942, 1350, 970, 5710, 557, 4314, 2715, 5371, 681, 1535, 2396, 4997, 4932, 4069, 5429, 5723, 3857, 3271, 6367, 858, 2223, 2939, 3336, 1768, 6111, 1837, 2090, 4384, 3745, 3108, 877, 1854, 111, 228, 1468, 265, 5594, 5967, 5367, 1208, 2928, 5728, 5128, 3085, 2892, 1896, 2180, 1127, 189, 3431, 5525, 5191, 2673, 3549, 925, 3298, 2558, 3885, 6167, 913, 2850, 3304, 2858, 612, 6102, 258, 712, 1876, 1283, 3168, 2517, 804, 115, 2643, 2487, 6383, 5016, 3746, 2275, 6541, 4166, 129, 6454, 3750, 4816, 2185, 1667, 3759, 1797, 1252, 3977, 3518, 2901, 2310, 3030, 2141, 6286, 2324, 4546, 633, 4701, 5703, 1105, 4198, 3732, 5136, 6212, 1324, 4083, 5554, 5472, 1340, 509, 2044, 5934, 4163,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 51 µs, sys: 1e+03 ns, total: 52 µs
Wall time: 54.4 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.54,0.46,999,0
1,-2.0,-1.95,test,0.64,0.36,0,0
2,-2.0,-1.90,test,0.70,0.30,0,0
3,-2.0,-1.85,test,0.73,0.27,0,0
4,-2.0,-1.80,test,0.82,0.18,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.84,0.16,0,0
6557,2.0,1.85,test,0.75,0.25,0,0
6558,2.0,1.90,test,0.69,0.31,0,0


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  656100


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,1,test
2,0,999,1,test
3,0,999,1,test
4,0,999,0,test
...,...,...,...,...
656095,6560,999,0,test
656096,6560,999,1,test
656097,6560,999,1,test
656098,6560,999,1,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9712230215827338,
 'eigenvector_val': 0.94604,
 'eigenvector_test': 0.823436018957346,
 'hamiltonian_train': 0.9892086330935251,
 'hamiltonian_val': 0.9852,
 'hamiltonian_test': 1.0}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}